# W207 Summer 2017 Final Project

## Personalized Medicine: Redefining Cancer Treatment



#### Matt Shaffer https://github.com/planetceres 
#### Kaggle Competition: https://www.kaggle.com/c/msk-redefining-cancer-treatment

According to [discussion boards](https://www.kaggle.com/c/msk-redefining-cancer-treatment/discussion/35810#202604) on Kaggle, the classes we are trying to predict appear to be as follows:

1. Likely Loss-of-function
2. Likely Gain-of-function
3. Neutral
4. Loss-of-function
5. Likely Neutral
6. Inconclusive
7. Gain-of-function
8. Likely Switch-of-function
9. Switch-of-function


#### Dependencies

In [1]:
import os
import time
import glob
import re
import pandas as pd
import numpy as np
import scipy.sparse as sps
import Bio

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import explained_variance_score
from sklearn.pipeline import make_pipeline

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.layers import Dropout
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping

%matplotlib inline
import matplotlib.pyplot as plt

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')
from itertools import islice

In [2]:
model_version = '001'

In [3]:
model_name = 'model_' + model_version

In [3]:
data_directory = '/Users/Reynard/dropbox/Data/kaggle/Personalized Medicine'
model_directory = data_directory + '/saved_models'

In [5]:
model_path = os.path.join(model_directory, model_name)

In [6]:
# Create model directory if it does not exist
if not os.path.isdir(model_directory):
    print("creating directory for saved models")
    os.mkdir(model_directory)

In [7]:
# Load model to resume training or perform inference
def load_model_from_json(model_path):
    model = model_from_json(open(model_path + '.json').read())
    model.load_weights(model_path + '.h5')
    #model.compile(optimizer=rmsprop, loss='mse')
    return model

In [8]:
from keras.models import load_model
# Load model to resume training or perform inference
def load_recent_model(model_path):
    # Locate the most recent model the folder to resume training from
    model_recent = max(glob.iglob(model_path + '*.hdf5'), key=os.path.getctime)
    print("Using model at checkpoint: {}".format(model_recent))
    #model = model_from_json(open(model_path + '.json').read())
    model = load_model(model_recent)
    #model.compile(optimizer=rmsprop, loss='mse')
    return model

In [9]:
# Save model
def save_model_to_json(m, model_path):    
    json_string = m.model.to_json()
    open(model_path + '.json', 'w').write(json_string)
    m.model.save_weights(model_path + '.h5', overwrite=True)

In [10]:
def print_op_str(data_type):
    p = "Done processing " + data_type + " data in {:.2f} seconds"
    return p

In [11]:
def print_blank(n):
    print(" "*n, end="\r")

### Data Overview

In [4]:
train_variants = pd.read_csv(data_directory + "/input/training_variants")
test_variants = pd.read_csv(data_directory + "/input/test_variants")
train_text = pd.read_csv(data_directory + "/input/training_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])
test_text = pd.read_csv(data_directory + "/input/test_text", sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"])

The test set has no labels and is used only for score submission. This will be a challenge since the sample size is small, and it will be hard to learn the properties of the population needed to perform inference. 

In [5]:
# Test set has no labels and is used 
print(list(train_variants.columns))
print(list(test_variants.columns))

['ID', 'Gene', 'Variation', 'Class']
['ID', 'Gene', 'Variation']


In addition to the gene variant data, we also have a text corpus for each example that provides the clinical evidence that human experts used to classify the genetic mutations. This is essentially an unstructured feature set, and our first task will be to map this noisy data to a set of features that can more easily be used for prediction. 

In [6]:
print(list(train_text.columns))
print(list(test_text.columns))

['ID', 'Text']
['ID', 'Text']


In [7]:
# Merge the text with the variant data, and separate the target values (`Class`) from the features
train = pd.merge(train_variants, train_text, how='left', on='ID')
y_train = train['Class'].values
X_train = train.drop('Class', axis=1)

In [8]:
# Do the same thing with the test data, but note that there are no classes to separate as targets
X_test = pd.merge(test_variants, test_text, how='left', on='ID')
test_index = X_test['ID'].values

In [9]:
# Create mini data sets for model building
train_mini = train.sample(frac=0.05)
y_train_mini = train_mini['Class'].values
X_train_mini = train_mini.drop('Class', axis=1)
X_test_mini = X_test.sample(frac=0.05)
test_index_mini = X_test_mini['ID'].values

# Create mini dev set for model building
dev_mini = train.sample(frac=0.05)
y_dev_mini = dev_mini['Class'].values
X_dev_mini = dev_mini.drop('Class', axis=1)

In [10]:
X_train_mini.shape

(166, 4)

In [12]:
# create dataset with all variants
all_variants = pd.concat([train_variants, test_variants], ignore_index=True)

Save/Load tokenized vocabulary

In [31]:
def save_sparse_csr(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

In [34]:
def load_sparse_csr(filename):
    loader = np.load(filename + '.npz')
    return sps.csr_matrix((  loader['data'], loader['indices'], loader['indptr']),
                         shape = loader['shape'])

In [51]:
save_sparse_csr(data_directory + '/data/train_bigram_vocabulary', tfidf_train)

CPU times: user 406 ms, sys: 599 ms, total: 1.01 s
Wall time: 1.23 s


In [52]:
save_sparse_csr(data_directory + '/data/test_bigram_vocabulary', tfidf_test)

CPU times: user 587 ms, sys: 797 ms, total: 1.38 s
Wall time: 1.63 s


Load tfidf from previous session if applicable:

In [35]:
%%time 
tfidf = TfidfVectorizer()
tfidf_train = load_sparse_csr(data_directory + '/data/train_bigram_vocabulary')
tfidf_test = load_sparse_csr(data_directory + '/data/test_bigram_vocabulary')

CPU times: user 911 ms, sys: 345 ms, total: 1.26 s
Wall time: 1.31 s


In order to reduce the dimensionality of the transformed text, we can use truncated singular value decomposition (SVD), which is similar to PCA, but for sparse matrices. 

In [36]:
# Choose index of number of features where we will truncate dimensionality
features_n_svd = 200

Load previously transformed data if applicable.

In [ ]:
svd_train = np.load(data_directory + '/data/train_svd_200.npy')
svd_test = np.load(data_directory + '/data/test_svd_200.npy')

In [37]:
%%time
svd = TruncatedSVD(features_n_svd, algorithm='arpack')
svd_train = svd.fit_transform(tfidf_train)
svd_test = svd.transform(tfidf_test)

In [38]:
svd_train.shape

(3321, 200)

In [45]:
np.save(data_directory + '/data/train_svd_200', svd_train)
np.save(data_directory + '/data/test_svd_200', svd_test)

### Fully Connected Neural Network

**  Hyperparameters **

In [60]:
input_shape = svd_train.shape[1]
output_shape = len(train['Class'].unique())
batch_n = 32
EPOCHS_N = 100
model_save_interval = 100

** Model **

This first attempt at a model is lossly based on encoder-decoder architecture where the degrees of freedom are iteratively restricted (i.e. hidden units => `512` => `256` => `128` => `64`), then inversely decoded (`64` => `128` => `256` => `512`).  Theoretically this might force the network to learn a compressed representation of the features. 

In [47]:
def model_hypothesis():
    model = Sequential()
    model.add(Dense(512, input_dim=input_shape, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(256, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_shape, kernel_initializer='normal', activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [48]:
def model_resume():
    model = load_recent_model(model_path)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

One-hot encoding of labels

In [55]:
# Callback for saving model and weights at n intervals in training
# https://keras.io/callbacks/#modelcheckpoint
weight_save_callback = ModelCheckpoint(model_path + '.{epoch:02d}-{loss:.2f}.hdf5', 
                                       monitor='loss', 
                                       verbose=0, 
                                       save_best_only=True, 
                                       mode='auto',
                                       period=model_save_interval # Interval (number of epochs) between checkpoints
                                      )

In [67]:
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=1, mode='auto')

In [58]:
onehot = LabelEncoder()
onehot.fit(y_train)
y_enc = onehot.transform(y_train)

In [59]:
y_ind = np_utils.to_categorical(y_enc)

To restore a previously saved model:

In [76]:
# Try to restore previous checkpoints to continue training
if os.path.isfile(model_path + '.h5') and os.path.isfile(model_path + '.json'):
    estimator = KerasClassifier(build_fn=model_resume, epochs=EPOCHS_N, batch_size=batch_n)
    model = model_resume()
else:
    estimator = KerasClassifier(build_fn=model_hypothesis, epochs=EPOCHS_N, batch_size=batch_n)
    model = model_hypothesis()

Using model at checkpoint: /Users/Reynard/dropbox/Data/kaggle/Personalized Medicine/saved_models/model_001.244-0.73.hdf5


Or create a new one:

In [ ]:
estimator = KerasClassifier(build_fn=model_hypothesis, epochs=EPOCHS_N, batch_size=batch_n)

In [77]:
%%time
start_time = time.time()
#estimator.fit(svd_train, y_ind, validation_split=0.05)
estimator.fit(svd_train, y_ind, batch_size=batch_n, epochs=EPOCHS_N*10, callbacks=[weight_save_callback, early_stopping])
end_time = time.time()
print("Elapsed time: {:.2f} sec".format(end_time-start_time))
try: 
    save_model_to_json(estimator, model_path)
    print("Saved model and weights to disk")
except Exception as e:
    print(e)

Using model at checkpoint: /Users/Reynard/dropbox/Data/kaggle/Personalized Medicine/saved_models/model_001.244-0.73.hdf5
Epoch 1/1000
3321/3321 [==============================] - 2s - loss: 0.7426 - acc: 0.7368     
Epoch 2/1000
3321/3321 [==============================] - 0s - loss: 0.7757 - acc: 0.7392     
Epoch 3/1000
3321/3321 [==============================] - 0s - loss: 0.7661 - acc: 0.7347     
Epoch 4/1000
3321/3321 [==============================] - 1s - loss: 0.7691 - acc: 0.7383     
Epoch 5/1000
3321/3321 [==============================] - 1s - loss: 0.7486 - acc: 0.7422     
Epoch 6/1000
3321/3321 [==============================] - 0s - loss: 0.7433 - acc: 0.7404     
Epoch 7/1000
3321/3321 [==============================] - 1s - loss: 0.7888 - acc: 0.7332     
Epoch 8/1000
3321/3321 [==============================] - 0s - loss: 0.7780 - acc: 0.7356     
Epoch 9/1000
3321/3321 [==============================] - 1s - loss: 0.7770 - acc: 0.7444     
Epoch 10/1000
3321/3321 

In [70]:
# Display model architecture
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model_hypothesis()).create(prog='dot', format='svg'))

# Save to .png
from keras.utils import plot_model
plot_model(model_hypothesis(), to_file='model_hypothesis1.png')

In [71]:
y_pred = estimator.predict_proba(svd_test)

5184/5668 [==========================>...] - ETA: 0s

In [73]:
submission = pd.DataFrame(y_pred)
submission['id'] = test_index
submission.columns = ['class1', 'class2', 'class3', 'class4', 'class5', 'class6', 'class7', 'class8', 'class9', 'id']
submission.to_csv(data_directory + "/output/submission_" + str(int(time.time())) + ".csv",index=False)